In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

import nltk
from nltk.corpus import stopwords 
import string
from nltk import word_tokenize

import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

In [57]:
f = open("/Users/dhams/Desktop/Research/PreProcessing/plot-genie-index-stop-removal.txt","r")
g = open("/Users/dhams/Desktop/Research/PreProcessing/clean-romance-pg.txt","r")
h = open("/Users/dhams/Desktop/Research/PreProcessing/adventure-pg-clean.txt","r")
i = open("/Users/dhams/Desktop/algo-detection/pre-processing/clean-argosy-1917.txt","r")
j = open("/Users/dhams/Desktop/algo-detection/pre-processing/clean-Dime-Detective.txt","r")
k = open("/Users/dhams/Desktop/algo-detection/pre-processing/clean-Popular-Detective.txt","r")
l = open("/Users/dhams/Desktop/algo-detection/pre-processing/clean-Science_Wonder_Stories.txt","r")
m = open("/Users/dhams/Desktop/algo-detection/pre-processing/clean-Terror_Tales.txt","r")

In [58]:
lines_1 = f.readlines()
lines_2 = g.readlines()
lines_3 = h.readlines()
lines_4 = i.readlines()
lines_5 = j.readlines()
lines_6 = k.readlines()
lines_7 = l.readlines()
lines_8 = m.readlines()
f.close()
g.close()
h.close()
i.close()
j.close()
k.close()
l.close()
m.close()
lines = lines_4 + lines_5 + lines_7 + lines_8 + lines_1 + lines_2 + lines_3 + lines_6

In [59]:
def send_tokens(line):
    tokens = word_tokenize(line)
    return tokens

In [60]:
from nltk.stem.wordnet import WordNetLemmatizer

def lemmatization(texts):
    lemmatizer = WordNetLemmatizer()
    lemmatized_word_list = []
    for line in texts:
        tokens = send_tokens(line)
        lemmatized_word_list.append([lemmatizer.lemmatize(w) for w in tokens])        
    return lemmatized_word_list

In [61]:
lemmatized_tokens = lemmatization(lines)

In [62]:
id2word = corpora.Dictionary(lemmatized_tokens)

# Create Corpus
texts = lemmatized_tokens

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]


In [75]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=8, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=10,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [76]:
topics = lda_model.print_topics()
doc_lda = lda_model[corpus]

In [77]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=lemmatized_tokens, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -11.79902169801584

Coherence Score:  0.388551557300689


In [78]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7      0.010240 -0.071669       1        1  16.141649
4     -0.089142  0.004823       2        1  15.939676
1      0.010904 -0.001042       3        1  13.217826
5      0.006289 -0.004178       4        1  12.825179
0      0.016805  0.018972       5        1  11.295601
3      0.014411  0.011970       6        1  11.050098
6      0.010218  0.012298       7        1   9.856715
2      0.020277  0.028826       8        1   9.673258, topic_info=      Category         Freq         Term        Total  loglift  logprob
term                                                                   
56     Default  1222.000000          one  1222.000000  30.0000  30.0000
117    Default  1054.000000          new  1054.000000  29.0000  29.0000
622    Default   820.000000          man   820.000000  28.0000  28.0000
46     Default   706.000000         hand   706.000000  27.0000  27.0000
815    Default   740.000000         room   740.000000  26.0000  26.0000
233    Default   718.000000         back   718.000000  25.0000  25.0000
700    Default   715.000000        money   715.000000  24.0000  24.0000
372    Default   679.000000         make   679.000000  23.0000  23.0000
65     Default   515.000000         made   515.000000  22.0000  22.0000
50     Default   586.000000         time   586.000000  21.0000  21.0000
527    Default   618.000000          car   618.000000  20.0000  20.0000
435    Default   476.000000         ring   476.000000  19.0000  19.0000
1415   Default   384.000000          air   384.000000  18.0000  18.0000
1917   Default   515.000000         face   515.000000  17.0000  17.0000
1337   Default   397.000000         name   397.000000  16.0000  16.0000
4835   Default   437.000000        radio   437.000000  15.0000  15.0000
1402   Default   366.000000         side   366.000000  14.0000  14.0000
15     Default   438.000000         home   438.000000  13.0000  13.0000
540    Default   351.000000        water   351.000000  12.0000  12.0000
60     Default   392.000000         good   392.000000  11.0000  11.0000
212    Default   304.000000          use   304.000000  10.0000  10.0000
604    Default   323.000000         must   323.000000   9.0000   9.0000
763    Default   338.000000       making   338.000000   8.0000   8.0000
158    Default   312.000000          way   312.000000   7.0000   7.0000
44     Default   331.000000         come   331.000000   6.0000   6.0000
1080   Default   366.000000         plan   366.000000   5.0000   5.0000
730    Default   314.000000         well   314.000000   4.0000   4.0000
1147   Default   304.000000         high   304.000000   3.0000   3.0000
67     Default   266.000000        small   266.000000   2.0000   2.0000
496    Default   287.000000       school   287.000000   1.0000   1.0000
...        ...          ...          ...          ...      ...      ...
26884   Topic8    31.579512      lambert   261.495697   0.2219  -6.6772
6362    Topic8    29.586590   obligation   244.866348   0.2224  -6.7424
4498    Topic8    19.975496        spark   163.115479   0.2359  -7.1352
22118   Topic8    23.823593         slot   196.542816   0.2256  -6.9591
6002    Topic8    33.229530  certificate   280.681671   0.2020  -6.6263
25638   Topic8    29.282988    chocolate   245.897476   0.2079  -6.7527
536     Topic8    26.087952     salesman   217.551865   0.2148  -6.8683
26821   Topic8    40.517208         mari   360.963318   0.1488  -6.4280
2291    Topic8    45.608292        motor   414.799438   0.1281  -6.3096
11906   Topic8    39.935699          kit   358.404175   0.1414  -6.4425
9827    Topic8    36.219894        mason   319.673828   0.1581  -6.5401
10269   Topic8    34.262722         tube   300.298157   0.1651  -6.5957
26774   Topic8    41.992111        isimo   389.854980   0.1075  -6.3923
3291    Topic8    28.487303       report   242.502991   0.1942  -6.7803
814     Topic8    64.966545      

In [120]:
unseen_document = 'How a Pentagon deal became an identity crisis for Google'
bow_vector = [id2word.doc2bow(text) for text in lemmatization(unseen_document)]
bow_vector_clean=[]
for i in bow_vector:
    if(i):
        bow_vector_clean.append(i)
for index, score in sorted(lda_model[bow_vector_clean], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

ValueError: too many values to unpack (expected 2)

In [122]:
unseen_document = 'How a Pentagon deal became an identity crisis for Google'
check = [string.split() for string in unseen_document]
bow_vector = id2word.doc2bow(unseen_document)

TypeError: doc2bow expects an array of unicode tokens on input, not a single string

In [112]:
c=[]
a=[[1,2],[],[4,5],[6],[7,8],[]]
 

In [114]:
c=[1,2]
a.append(c)

In [115]:
a

[[1, 2], [], [4, 5], [6], [7, 8], [], [1, 2]]

In [116]:
c=[]
for i in a:
    if(i):
        c.append(i)

In [117]:
c

[[1, 2], [4, 5], [6], [7, 8], [1, 2]]

In [118]:
bow_vector_clean=[]
for i in bow_vector:
    if(i):
        bow_vector_clean.append(i)

In [119]:
bow_vector_clean

[[(22610, 1)],
 [(3578, 1)],
 [(21891, 1)],
 [(24150, 1)],
 [(3578, 1)],
 [(24150, 1)],
 [(21891, 1)],
 [(3578, 1)],
 [(21891, 1)],
 [(3578, 1)],
 [(1287, 1)],
 [(21891, 1)],
 [(3578, 1)],
 [(24150, 1)],
 [(11044, 1)],
 [(21891, 1)],
 [(24150, 1)],
 [(11044, 1)],
 [(13267, 1)],
 [(11044, 1)],
 [(16124, 1)],
 [(11044, 1)],
 [(16124, 1)],
 [(21891, 1)]]